In [ ]:
from libreria import Graduate
from selenium.webdriver.common.keys import Keys
from datetime import datetime


In [ ]:
graduacion = Graduate()

In [ ]:
def format_fecha(fecha, from_csv = False):
    if not from_csv:
        if len(fecha) <= 5:
            fecha = datetime.strptime(fecha + '/23', '%d/%m/%y').strftime('%d/%m/%Y')
        else:
            fecha = datetime.strptime(fecha, '%d/%m/%y').strftime('%d/%m/%Y')
        return fecha
    else:
        fecha = datetime.strptime(fecha, '%d/%m/%Y').strftime('%d/%m/%Y')
        return fecha
    
def tabla(n):
    tabla = f'//*[@id="MainContent_gvBenchmark{n}"]/tbody'
    return tabla

def elemento(ruta):
    elemento = graduacion.elemento(ruta)
    return elemento

In [ ]:
def main():
    
    graduacion.abrir_formulario()
    graduacion.set_fecha(format_fecha(input('Fecha de evaluacion')))
    graduacion.select_hogar(input('Codigo del hogar'))

    previousy_assessed = input('Previamente evaluado?')
    graduacion.seleccionar('//*[@id="MainContent_ddlHHPreviouslyAssessed"]',previousy_assessed)
    if str(previousy_assessed) == '1':
        elemento('//*[@id="MainContent_txtDatePreviouslyAssessed"]').send_keys\
            (format_fecha(input('Fecha en que fue evaluado')), Keys.ENTER)

    # Punto 1

    beneficiarios = graduacion.extraer_cantidad(tabla(1))

    for i in range(beneficiarios):
        r = input('Fecha en la que se reporta estatus')
        if r == '1': continue
        elif r == '0': break
        else:
            elemento(f'//*[@id="MainContent_gvBenchmark1_txtDateHIVStatusDocumented_{i}"]')\
                .send_keys(format_fecha(r), Keys.ENTER)
            
    # Punto 2

    for i in range(graduacion.extraer_cantidad(tabla(2))):
        p2a = f'//*[@id="MainContent_gvBenchmark2_ddlAttendingARTAppointments_{i}"]'
        p2b = f'//*[@id="MainContent_gvBenchmark2_ddlTakingARTPillsAsPrescribed_{i}"]'
        if elemento(p2a).is_enabled():
            r = input('Digitar respuestas para punto 2')
            graduacion.seleccionar(p2a, r[0])
            graduacion.seleccionar(p2b, r[1])

    # Punto 3

    beneficiarios = graduacion.extraer_cantidad(tabla(3)) / 22
    if int(beneficiarios) != 0:
        for b in range(int(beneficiarios)): # Para repetir por la cantidad de beneficiarios
            respuestas = 0
            
            r = input('Pregunta 3.1')
            for i in range(len(str(r))): # Para repetir en base al input a la pregunta 3.1
                elemento(f'//*[@id="MainContent_gvBenchmark3_cblWaysOfGettingHIV_{b}_{r[i]}_{b}"]').click()
                respuestas += 1

            r = input('Pregunta 3.2')
            for i in range(len(str(r))): # Para repetir en base al input a la pregunta 3.2
                elemento(f'//*[@id="MainContent_gvBenchmark3_cblWaysOfProtectingAgainstHIV_{b}_{r[i]}_{b}"]').click()
                respuestas += 1

            r = input('Pregunta 3.3')
            for i in range(len(str(r))): # Para repetir en base al input a la pregunta 3.3
                elemento(f'//*[@id="MainContent_gvBenchmark3_cblWhereHIVPreventionSupport_{b}_{r}_{b}"]').click()
                if str(r[i]) == '4':
                    elemento(f'//*[@id="MainContent_gvBenchmark3_txtWhereHIVPreventionSupportOther_{b}"]')\
                        .send_keys(input('Otro'))
                respuestas += 1
    
            if respuestas > 4:
                graduacion.seleccionar(f'//*[@id="MainContent_gvBenchmark3_ddlChildAwareOfHIVRisksAndPreventionMeasures_{b}"]', 1)
            else:
                graduacion.seleccionar(f'//*[@id="MainContent_gvBenchmark3_ddlChildAwareOfHIVRisksAndPreventionMeasures_{b}"]', 2)
    # Punto 4

    children1 = graduacion.extraer_cantidad(tabla(4))
    children2 = graduacion.extraer_cantidad(tabla('4b'))

    if children1 != 0:
        for i in range(children1):
            graduacion.seleccionar(f'//*[@id="MainContent_gvBenchmark4_ddlChildMUACMoreThan12_5_{i}"]', 1)
            graduacion.seleccionar(f'//*[@id="MainContent_gvBenchmark4_ddlChildFreeOfAnySignsOfBipedalEdema_{i}"]',1)

    if children2 != 0:
        for i in range(children2):
            print('agregar rutas')  

    # Punto 5 

    P = ('//*[@id="MainContent_ddlAbleToPayForMedicalCosts"]',
        '//*[@id="MainContent_ddlAbleToPayForMedicalCostsWithoutCashTransfer"]',
        '//*[@id="MainContent_ddlAbleToPayForMedicalCostsWithoutSellingAnything"]',
        '//*[@id="MainContent_ddlAbleToPayForSchoolSupplies"]',
        '//*[@id="MainContent_ddlAbleToPayForSchoolSuppliesWithoutFinancialAid"]',
        '//*[@id="MainContent_ddlAbleToPayForSchoolSuppliesWithoutSellingAnything"]')

    r = input('Punto 5')
    if r == '1a':
        for i in range(6):
            try:
                graduacion.seleccionar(P[i], 1)
            except NotImplementedError: break
    elif r == '2a':
        for i in range(6):
            try:
                graduacion.seleccionar(P[i], 2)
            except NotImplementedError: break
    else:
        for i in range(len(str(r))):
            graduacion.seleccionar(P[i],int(r[i]))

    # Punto 6 a 7 

    P = ('//*[@id="MainContent_ddlAwareOfAnyPhysicalAbuse"]',
            '//*[@id="MainContent_ddlAwareOfAnySexualAbuse"]',
            '//*[@id="MainContent_ddlChildrenUnderCareOfStableAdultCaregiver"]')

    for i in range(3):
        if i < 2:
            graduacion.seleccionar(P[i], 2)
        else:
            graduacion.seleccionar(P[i], 1)

    # P8

    def format_preguntas(n):
        P = (f'//*[@id="MainContent_gvBenchmark8_ddlChildEnrolledInSchool_{n}"]',
            f'//*[@id="MainContent_gvBenchmark8_ddlChildAttendedSchoolRegularly_{n}"]',
            f'//*[@id="MainContent_gvBenchmark8_ddlChildProgressedToNextGrade_{n}"]',
            f'//*[@id="MainContent_gvBenchmark8_ddlPCGAnswerYesToAllBM8_{n}"]')
        return P

    beneficiarios = graduacion.extraer_cantidad(tabla(8))
    if beneficiarios != 0:
        for b in range(beneficiarios):
            for p in range(4):
                graduacion.seleccionar(format_preguntas(b)[p], 1)


In [ ]:
for i in range(50):
    main()
    if input('Desea continuar?') == '':
        elemento('//*[@id="MainContent_btnHHInfo"]').click()
    graduacion.esperar_alerta()


In [ ]:
graduacion.cerrar()